In [1]:
import torch
import mlflow
import mlflow.pytorch
import torch.nn as nn
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.functional as F

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
Max = 30
class Transliteration_Encoder_Decoder(nn.Module):
  
  def __init__(self, input_size, hidden_size, output_size):
    super(Transliteration_Encoder_Decoder,self).__init__()

    self.hidden_size = hidden_size
    self.output_size = output_size

    self.encoder_rnn_cell = nn.GRU(input_size, hidden_size)
    self.decoder_rnn_cell = nn.GRU(hidden_size*2, hidden_size)

    self.h2o = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=2)

    self.U = nn.Linear(self.hidden_size,self.hidden_size)
    self.W = nn.Linear(self.hidden_size,self.hidden_size)
    self.attn = nn.Linear(self.hidden_size, 1)
    self.out2hidden = nn.Linear(self.output_size, self.hidden_size)


  def forward(self, input_, max_char = Max, device = 'cpu', gt=None):
    # encoder
    eout, hidden = self.encoder_rnn_cell(input_)
    eout = eout.view(-1,self.hidden_size)
    # decoder
    decoder_state = hidden
    decoder_input = torch.zeros(1, 1, self.output_size).to(device)
   
    outputs = []
    U = self.U(eout)

    for i in range(max_char):

      W = self.W(decoder_state.view(1,-1).repeat(eout.shape[0], 1))
      V = self.attn(torch.tanh(U + W))
      attn_weights = F.softmax(V.view(1,-1), dim = 1)

      attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                               eout.unsqueeze(0))

      embedding = self.out2hidden(decoder_input)
      decoder_input = torch.cat((embedding[0], attn_applied[0]),1).unsqueeze(0)

      out , decoder_state = self.decoder_rnn_cell(decoder_input, decoder_state)
      
      out = self.h2o(decoder_state)
      out = self.softmax(out)
      outputs.append(out.view(1,-1))

      #giving output of one nn to next nn

      idx= torch.argmax(out,2,keepdim=True)
      if not gt is None:
        idx = gt[i].reshape(1,1,1)
      one_hot = torch.zeros(out.shape, device = device)
      one_hot.scatter_(2,idx,1)

      decoder_input = one_hot.detach()
    return outputs

In [4]:
model0 = torch.load("model1.pt")

In [5]:
model1 = mlflow.pytorch.load_model("mlflow-artifacts:/0/471282cbeafc4734b6ec1969bf72d4bb/artifacts/models")

In [6]:
model3 = mlflow.pytorch.load_model("mlflow-artifacts:/0/ac6b9dfcd8794900802530f9e7b955b3/artifacts/models")

In [7]:
model4 = mlflow.pytorch.load_model("mlflow-artifacts:/0/cc7895f4ad3b4d15ac5a2a66e709295f/artifacts/models")

In [8]:
model5 = mlflow.pytorch.load_model("mlflow-artifacts:/0/e0e502fbf28e4a248089fd40df764480/artifacts/models")

In [9]:
model2 = mlflow.pytorch.load_model("mlflow-artifacts:/0/09f32059eeb24723acefb00d2bb418fa/artifacts/models")

In [10]:
model6 = mlflow.pytorch.load_model("mlflow-artifacts:/0/be75d328439841529d305948b6b1b14a/artifacts/models")

In [11]:
eng_alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
pad = '-PAD-'
engdic = {pad : 0}
for index ,alpha in enumerate(eng_alphabets):
  engdic[alpha] = index+1
print(engdic)

{'-PAD-': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [12]:
tel_alphabets = [chr(alpha) for alpha in range(3072,3199)]
tel_alphabets.append(chr(8204))
tel_alphabets.append(chr(8205))
tel_size = len(tel_alphabets)
teldic = {pad : 0}
for index ,alpha in enumerate(tel_alphabets):
  teldic[alpha] = index+1
print(teldic)  


{'-PAD-': 0, 'ఀ': 1, 'ఁ': 2, 'ం': 3, 'ః': 4, 'ఄ': 5, 'అ': 6, 'ఆ': 7, 'ఇ': 8, 'ఈ': 9, 'ఉ': 10, 'ఊ': 11, 'ఋ': 12, 'ఌ': 13, '\u0c0d': 14, 'ఎ': 15, 'ఏ': 16, 'ఐ': 17, '\u0c11': 18, 'ఒ': 19, 'ఓ': 20, 'ఔ': 21, 'క': 22, 'ఖ': 23, 'గ': 24, 'ఘ': 25, 'ఙ': 26, 'చ': 27, 'ఛ': 28, 'జ': 29, 'ఝ': 30, 'ఞ': 31, 'ట': 32, 'ఠ': 33, 'డ': 34, 'ఢ': 35, 'ణ': 36, 'త': 37, 'థ': 38, 'ద': 39, 'ధ': 40, 'న': 41, '\u0c29': 42, 'ప': 43, 'ఫ': 44, 'బ': 45, 'భ': 46, 'మ': 47, 'య': 48, 'ర': 49, 'ఱ': 50, 'ల': 51, 'ళ': 52, 'ఴ': 53, 'వ': 54, 'శ': 55, 'ష': 56, 'స': 57, 'హ': 58, '\u0c3a': 59, '\u0c3b': 60, '\u0c3c': 61, 'ఽ': 62, 'ా': 63, 'ి': 64, 'ీ': 65, 'ు': 66, 'ూ': 67, 'ృ': 68, 'ౄ': 69, '\u0c45': 70, 'ె': 71, 'ే': 72, 'ై': 73, '\u0c49': 74, 'ొ': 75, 'ో': 76, 'ౌ': 77, '్': 78, '\u0c4e': 79, '\u0c4f': 80, '\u0c50': 81, '\u0c51': 82, '\u0c52': 83, '\u0c53': 84, '\u0c54': 85, 'ౕ': 86, 'ౖ': 87, '\u0c57': 88, 'ౘ': 89, 'ౙ': 90, 'ౚ': 91, '\u0c5b': 92, '\u0c5c': 93, '\u0c5d': 94, '\u0c5e': 95, '\u0c5f': 96, 'ౠ': 97, 'ౡ': 98, 'ౢ': 99, 

In [13]:
def gt_rep(word, dic, device = 'cpu'):
  word = word.upper()
  rep = torch.zeros([len(word)+1, 1], dtype=torch.long).to(device)
  for letter_index, letter in enumerate(word):
    pos = dic[letter]
    rep[letter_index][0] = pos
  rep[letter_index+1][0] = dic[pad]
  return rep

In [14]:
def word_rep(word, dic, device = 'cpu'):
  word_rep = torch.zeros(len(word)+1, 1, len(dic)).to(device)
  for letter_index , letter in enumerate(word):
    pos = dic[letter]
    word_rep[letter_index][0][pos] = 1
  pad_pos = dic[pad]
  word_rep[letter_index+1][0][pad_pos] = 1
  return word_rep  

In [15]:
def infer(net, name,n, device):
  net.eval().to(device)
  name_ohe = word_rep(name,teldic,device)
  output = net(name_ohe,n,device)
  return output

In [16]:
out = infer(model3 , "తస్కత్సం",20,device_gpu)

In [17]:
sample = word_rep("పసునూరి",teldic)
print(sample.size())

torch.Size([8, 1, 130])


In [18]:
for i in range(len(out)):
  print(out[i].shape, list(engdic.keys())[list(engdic.values()).index(torch.argmax(out[i]))])

torch.Size([1, 27]) T
torch.Size([1, 27]) A
torch.Size([1, 27]) S
torch.Size([1, 27]) K
torch.Size([1, 27]) A
torch.Size([1, 27]) T
torch.Size([1, 27]) S
torch.Size([1, 27]) A
torch.Size([1, 27]) M
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-
torch.Size([1, 27]) -PAD-


In [21]:
a="తెలుగు అనేది ద్రావిడ భాషల కుటుంబానికి చెందిన భాష దీనిని మాట్లాడే ప్రజలు ప్రధానంగా ఆంధ్ర తెలంగాణాలో ఉన్నారు ఇది ఆ రాష్ట్రాలలో అధికార భాష భారతదేశంలో ఒకటి కంటే ఎక్కువ రాష్ట్రాల్లో ప్రాథమిక అధికారిక భాషా హోదా కలిగిన కొద్ది భాషలలో హిందీ బెంగాలీలతో పాటు ఇది కూడా ఉంది భారతదేశంలో అత్యధికంగా మాతృభాషగా మాట్లాడే భాషలలో తెలుగు నాలుగో స్థానంలో ఉంది భారత దేశంలో జనాభా లెక్కల ప్రకారం దాదాపు మిలియన్ల మంది మాట్లాడేవారున్నారు ప్రపంచ వ్యాప్తంగా మాతృభాషగా మాట్లాడే భాషల ఎథ్నోలాగ్ జాబితాలో వ స్థానంలో ఉంది ఇది ద్రావిడ భాషా కుటుంబంలో ఎక్కువమంది మాట్లాడే భాష భారతదేశంలో ఇరవై రెండు షెడ్యూల్ భాషలలో ఇది ఒకటి ఇది అమెరికాలో వేగంగా అభివృద్ధి చెందుతున్న భాష తెలుగు భాషలో సుమారు "
words=a.split()
s=""
for word in words:
  out2 = infer(model6,word,10,device_gpu)
  for i in range(len(out2)):
    k=(list(engdic.keys())[list(engdic.values()).index(torch.argmax(out2[i]))])
    if(str(k)=="-PAD-"):
      continue
    s=s+str(k)
  s=s+" "
print(s)
   

TELUGU ANEDI DRAVIDA BHASHALA KUTUMBANIK CHENDINA BHASHA DENINI MATLADE PRAJALU PRADHANAGA ANDHRA TELANGANAL UNNARU IDI AA RASHTRALLL ADHIKARA BHASHA BHARATADES OKATI KANTE EKKUVA RASHTRALLO PRATHAMIKA ADHIKARIKA BHASHA HODA KALIGINA KODDI BHASHALLLO HINDI BENGALILAT PATU IDI KUDDA UNDI BHARATADES ATYADIIKAN MATHRBHASH MATLADE BHASHALLLO TELUGU NALUGO STHANALOO UNDI BHARATA DESAALOO JANABHA LEKKALA PRAKARAM DADAPU MILIANLA MANDI MATLADEVAR PRAPANCHA VYATTANGA MATHRBHASH MATLADE BHASHALA ETHNOLGG JABITALO VA STHANALOO UNDI IDI DRAVIDA BHASHA KUTUMBAMLO EKKUVAMMDI MATLADE BHASHA BHARATADES IRAVI RENDU SHEDULL BHASHALLLO IDI OKATI IDI AMERIKALOO VEGANGA ABHIVRDDHI CHENDUTUNN BHASHA TELUGU BHASHALO SUMARU 


In [ ]:
6TELUGU ANEDI DRAVIDA BHASHALA KUTUMBANIK CHENDINA BHASHA DENINI MATLADE PRAJALU PRADHANAGA ANDHRA TELANGANAL UNNARU IDI AA RASHTRALLL ADHIKARA BHASHA BHARATADES OKATI KANTE EKKUVA RASHTRALLO PRATHAMIKA ADHIKARIKA BHASHA HODA KALIGINA KODDI BHASHALLLO HINDI BENGALILAT PATU IDI KUDDA UNDI 

In [ ]:
3TELLUGU ANEDI DRAVIDD BHASHALA KUTUMBANIK CHENDINA BHASHA DEENIII MATLADE PRAJALU PRADHANANG ANDHRA TELLANGANA UNNARU IDI AA RASHTRALLL ADHIKARA BHASHA BHARATADES OKATI KANTE EKKUVA RASHTRALLO PRATHAMIKA ADHIKARIKK BHASHA HODA KALIGINA KODDI BHASHALLLO HINDI BENGALILAT PATU IDI KUDA UNDI

In [ ]:
2TELUGU ANEDI DRAVIDA BHASHALA KUTUMBANII CHENDINA BHASHA DEENNI MATLADE PRAJALU PRADHANAGA ANDHRA TELANGANAL UNNARU IDI AA RASHTRALLL ADHIKARA BHASHA BHARATHETA OKATI KANTE EKKUVA RASHTRALLO PRATHAMIKA ADHIKARIKA BHASHA HODA KALIGINA KODDI BHASHALLLO HINDI BENGALILAT PATU IDI KUDA UNDI 

In [ ]:
5TELLUGU ANEDI DRAVIDA BHASHALA KUTUNBANII CHENDINA BHASHA DEENINI MATLADE PRAJALU PRADHANANG ANDHRA TELANGANAL UNNAR IDI AA RASHTRALLL ADHIKARA BHASHA BHARATADES OKATI KANTE EKKUVA RASHTRALLO PRATHAMIKA ADHIKARIKA BHASHA HODA KALIGINA KODDI BHASHALLLO HINDI BENGALILAT PATU IDI KUDA UNDI 

In [8]:
traced_model = torch.jit.trace(model.to(torch.device("cpu")), torch.randn(20, 1, 130).to("cpu"))
traced_model.save('Androidmodel.pt')

In [18]:
print(type(out))

<class 'list'>


In [19]:
print(out)

[tensor([[-11.6033,  -5.1333, -11.8941,  -7.0798,  -6.5921,  -7.5096,  -9.4377,
          -7.3692,  -6.2707,  -7.7255, -12.5182,  -9.0643,  -9.6306,  -9.5077,
          -7.4743, -10.6196,  -7.2057, -10.6306,  -7.0185,  -6.2257,  -0.0168,
          -8.9025,  -8.5771,  -8.7973, -12.5460, -10.9065, -12.5018]],
       device='cuda:0', grad_fn=<ViewBackward0>), tensor([[ -9.7788,  -0.1689, -13.9893,  -9.6168, -11.4963,  -5.7406, -16.0995,
         -13.1351,  -1.9992,  -6.3859, -14.4776, -11.3705,  -9.5609, -11.5237,
          -9.7957,  -5.8482, -12.0105, -15.5033,  -8.8353,  -7.2346,  -5.8312,
          -4.8941, -10.9334,  -9.1833, -15.3911,  -7.7335, -14.1730]],
       device='cuda:0', grad_fn=<ViewBackward0>), tensor([[ -8.5035,  -2.8494,  -7.7627,  -5.2146,  -9.9531,  -6.7927, -11.5295,
          -9.5764,  -7.6060,  -6.5669,  -7.4923,  -7.9452,  -6.8112,  -7.9977,
          -6.3251,  -7.4502, -10.5762, -10.0070,  -7.7280,  -0.1384,  -5.7746,
          -3.0043,  -7.2429,  -7.1359,  -6.738

In [33]:
androidmodel = torch.jit.load("Androidmodel1.pt")


In [34]:
print(str(word_rep("తెలుగు",teldic).size()))

torch.Size([7, 1, 130])


In [40]:
output = androidmodel(word_rep("పసునూరి",teldic))

In [36]:
print(type(output[0]))

<class 'torch.Tensor'>


In [38]:
s = " "
for i in range(len(output)):
    k = list(engdic.keys())[list(engdic.values()).index(torch.argmax(output[i]))]
    if(k==pad):
        continue;
    else:
        s=s+k
print(s)        

 PASUNURI


In [ ]:
torch.Tensor()